# <h2 align=center> PROYECTO INDIVIDUAL Nº1 </h2>
# <h1 align=center>Recomendacion de Películas</h1>
        @autor: Lesmen Gerardo García Pernia

## Objetivos:

+ Crear un Modelo de Sistema de Recomendacion utilizando las caracteristicas de las películas para encontrar peliculas similares a las que ha visto un determinado usuario y le han gustado.

Estaremos utilizando un Sistema de Recomendacion Basado en Contenido

Definición de librerias

In [146]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
from typing import Optional   # El Tipo del atributo podria ser opcional

from fastapi.responses import FileResponse
from fastapi.templating import Jinja2Templates
from fastapi.responses import JSONResponse

import pandas as pd
import re
import webbrowser
import textwrap

import traceback
import logging

import json
import ast

Fuente de datos

+ [Dataset](https://drive.google.com/drive/folders/1mfUVyP3jS-UMdKHERknkQ4gaCRCO2e1v): <Br>
Dos (2) Archivos de datos: movies_dataset.csv y credits.csv
+ [Diccionario de datos](https://docs.google.com/spreadsheets/d/1QkHH5er-74Bpk122tJxy_0D49pJMIwKLurByOfmxzho/edit#gid=0): <br/>
Diccionario con algunas descripciones de las columnas disponibles en el dataset.


## 1) Inicio del proceso de ETL

### Cargar los 2 archivos de datos: movies_dataset.csv y credits.csv

In [5]:
df_movies = pd.read_csv('../DataSetsPI/movies_dataset.csv')
df_credits = pd.read_csv('../DataSetsPI/credits.csv')

C:\Users\lesme\AppData\Local\Temp\ipykernel_1820\1434914061.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('../DataSetsPI/movies_dataset.csv')


### 1.1) ETL al DataSets: Movies:

In [6]:
### Validar los tipos de datos de las columnas y cantidad total:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [7]:
### Visualizar el dataSets de movies:
df_movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [8]:
### Validar las columnas numericas definidas inicialmente:
df_movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


# Se procede a realizar los requerimiento iniciales del proyecto.

### Requerimiento Nro. 1

+ Algunos campos, como **`belongs_to_collection`**, **`production_companies`** y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder  y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [9]:
### Visualizando la primera fila del DataSets
df_movies.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

### 1) Visualizando la columna: *`belongs_to_collection`*

In [10]:
### Tomando el primer dato de la columna 'belongs_to_collection'
df_movies['belongs_to_collection'][0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

Podemos observar que los datos son String (Object) pero que tienen un formato tipo diccionario. De tal manera, que en este caso nos interesa extraer la clave-valor ⏩ `'name': 'Toy Story Collection'`, ya que corresponde a que franquicia o series de películas pertenece la película.

### 2) Visualizando la columna: *`genres`*

In [11]:
### Tomando el primer dato de la columna 'belongs_to_collection'
df_movies['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

Podemos observar que los datos son String (Object) pero que tienen un formato de una lista con varios diccionarios. De tal manera, que en este caso nos interesa extraer la clave-valor ⏩ `'name': 'Animation', 'name': 'Comedy', 'name': 'Family'`, ya que corresponden a todos los generos asociados a la película.

### 3) Visualizando la columna: *`production_companies`*

In [12]:
### Tomando el primer dato de la columna 'belongs_to_collection'
df_movies['production_companies'][0]

"[{'name': 'Pixar Animation Studios', 'id': 3}]"

### 4) Visualizando la columna: *`production_countries`*

In [13]:
### Tomando el primer dato de la columna 'belongs_to_collection'
df_movies['production_countries'][0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

Podemos observar que los datos son String (Object) pero que tienen un formato de una lista con un diccionari. De tal manera, que en este caso nos interesa extraer la clave-valor ⏩ `'name': 'United States of America`, ya que corresponden a una lista con los paises donde se produjo la película.

### 5) Visualizando la columna: *`spoken_languages`*

In [14]:
### Tomando el primer dato de la columna 'belongs_to_collection'
df_movies['spoken_languages'][0]

"[{'iso_639_1': 'en', 'name': 'English'}]"

Podemos observar que los datos son String (Object) pero que tienen un formato de una lista con un diccionari. De tal manera, que en este caso nos interesa extraer la clave-valor ⏩ `'name': 'English`, ya que corresponden a los idiomas que se hablan en la película.

In [15]:
### Se visualiza mejor el escenario de las columnas anidadas
for i in (1,3,12,13,17):
    print(df_movies.iloc[4][i])

{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}
[{'id': 35, 'name': 'Comedy'}]
[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_639_1': 'en', 'name': 'English'}]


### Se procede a desanidar o separar el valor de la clave:'name' por ser el dato de mayor relevancia en estos campos; de las siguientes columnas:

<ul>
<li> 1) belongs_to_collection (Diccionario).</li>
<li> 2) genres (Lista de Diccionario). </li>
<li> 3) production_companies (Lista de Diccionario).</li>
<li> 4) production_countries (Lista de Diccionario).</li>
<li> 5) spoken_languages (Lista de Diccionario).</li>
</ul>

### Creación de Funciones para desanidar las columnas indicadas anteriormente:

Se procede a crear dos Funciones para Desanidar estas 5 columnas anidadas. La primera función ***(`Extraer_Valor_Dicc`)***, para extraer de la columna 'belongs_to_collection' sus campos en forma de diccionarios. Y la segunda función ***(`Extraer_Valor_List`)***, para extraer del resto de las columnas que contienen sus campos como una lista de diccionarios.

1 ) Funcion: `Extraer_Valor_Dicc`:<Br>
*Funcion para evaluar columnas tipo Diccionario y extraer el valor asociado a la clave 'name'; devolviendo dicho valor.*

In [16]:
# Función para extraer el valor de la clave 'name', de la columna diccionario: 
def Extraer_Valor_Dicc(value):
    try:
        diccionario = ast.literal_eval(value)  #Evaluamos cada atributo como un diccionario de Python
        if isinstance(diccionario, dict): #Si el valor es un diccionario, extrae el valor asociado a la clave 'name', para luego retornarlo
            return diccionario.get('name', '')
    except (ValueError, SyntaxError): #Si no es posible evaluarlo como un diccionario, capturamos las excepciones (ValueError o SyntaxError) y devolvemos una cadena vacía.
        pass
    return ''  # En caso de no ser un diccionario, se devuelve una cadena vacía.

2  ) Funcion: `Extraer_Valor_Lista`:<Br>
*Funcion para evaluar columnas tipo Listas de Diccionario y extraer el valor asociado a la clave 'name' de cada diccionario, concatenándolos en una sola cadena.* 

In [17]:
# Función para extraer el valor de la clave 'name', de la columna lista de diccionarios: genres
def Extraer_Valor_List(value):
    try:
        lista_de_diccionarios = ast.literal_eval(value) ##Evaluamos cada atributo como una lista de diccionario de Python
        if isinstance(lista_de_diccionarios, list): #Si el valor es una lista de diccionario, extrae el valor asociado a la 
                                                    #clave 'name' de cada diccionario y lo concatena en una sola cadena.
            names = [dic.get('name', '') for dic in lista_de_diccionarios if isinstance(dic, dict)]
            return '|'.join(names)
    except (ValueError, SyntaxError):
        pass
    return ''  # Si no es posible evaluarlo como una lista de diccionarios, se devuelve una cadena vacía.

### Ejecución de las funciones para desanidar las columnas siguientes:

**`1) belongs_to_collection`**

In [18]:
# Aplicar la función para extraer el valor 'name' y crear la nueva columna 'Collection'
df_movies['collection'] = df_movies['belongs_to_collection'].apply(Extraer_Valor_Dicc)
df_movies['collection']

0                  Toy Story Collection
1                                      
2             Grumpy Old Men Collection
3                                      
4        Father of the Bride Collection
                      ...              
45461                                  
45462                                  
45463                                  
45464                                  
45465                                  
Name: collection, Length: 45466, dtype: object

**`2) genes`**

In [19]:
# Aplicar la función para extraer los valores 'name' y crear la nueva columna 'genres'
df_movies['genre'] = df_movies['genres'].apply(Extraer_Valor_List)
df_movies['genre']

0         Animation|Comedy|Family
1        Adventure|Fantasy|Family
2                  Romance|Comedy
3            Comedy|Drama|Romance
4                          Comedy
                   ...           
45461                Drama|Family
45462                       Drama
45463       Action|Drama|Thriller
45464                            
45465                            
Name: genre, Length: 45466, dtype: object

**`3) production_companies`**

In [20]:
# Aplicar la función para extraer los valores 'name' y crear la nueva columna 'Collection'
df_movies['companies'] = df_movies['production_companies'].apply(Extraer_Valor_List)
df_movies['companies']

0                                  Pixar Animation Studios
1        TriStar Pictures|Teitler Film|Interscope Commu...
2                              Warner Bros.|Lancaster Gate
3                   Twentieth Century Fox Film Corporation
4                Sandollar Productions|Touchstone Pictures
                               ...                        
45461                                                     
45462                                          Sine Olivia
45463                              American World Pictures
45464                                            Yermoliev
45465                                                     
Name: companies, Length: 45466, dtype: object

**`4) production_countries`**

In [21]:
# Aplicar la función para extraer los valores 'name' y crear la nueva columna 'Collection'
df_movies['countries'] = df_movies['production_countries'].apply(Extraer_Valor_List)
df_movies['countries']

0        United States of America
1        United States of America
2        United States of America
3        United States of America
4        United States of America
                   ...           
45461                        Iran
45462                 Philippines
45463    United States of America
45464                      Russia
45465              United Kingdom
Name: countries, Length: 45466, dtype: object

**`5) spoken_languages`**

In [22]:
# Aplicar la función para extraer los valores 'name' y crear la nueva columna 'Collection'
df_movies['languages'] = df_movies['spoken_languages'].apply(Extraer_Valor_List)
df_movies['languages']

0                 English
1        English|Français
2                 English
3                 English
4                 English
               ...       
45461               فارسی
45462                    
45463             English
45464                    
45465             English
Name: languages, Length: 45466, dtype: object

### Validando las nuevas 5 columnas creadas

In [23]:
### Validando las nuevas 5 columnas creadas
df_movies[['collection', 'genre', 'companies', 'countries', 'languages' ]]

,collection,genre,companies,countries,languages
0,Toy Story Collection,Animation|Comedy|Family,Pixar Animation Studios,United States of America,English
1,,Adventure|Fantasy|Family,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,English|Français
2,Grumpy Old Men Collection,Romance|Comedy,Warner Bros.|Lancaster Gate,United States of America,English
3,,Comedy|Drama|Romance,Twentieth Century Fox Film Corporation,United States of America,English
4,Father of the Bride Collection,Comedy,Sandollar Productions|Touchstone Pictures,United States of America,English
...,...,...,...,...,...
45461,,Drama|Family,,Iran,فارسی
45462,,Drama,Sine Olivia,Philippines,
45463,,Action|Drama|Thriller,American World Pictures,United States of America,English
45464,,,Yermoliev,Russia,


### Se procede a eliminar las 5 Columnas que se estan anidadas:

In [24]:
### Se procede a eliminar las 5 Columnas que se estan anidadas:
df_movies.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 
                        'production_countries',  'spoken_languages' ], inplace=True)

### Validan nuevamente el total de registros y columnas:
df_movies.shape

(45466, 24)

In [25]:
### Listando todas las columnas
df_movies.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'collection', 'genre',
       'companies', 'countries', 'languages'],
      dtype='object')

### Requerimiento Nro. 2

+ Los valores nulos de los campos **`revenue`**, **`budget`** deben ser rellenados por el número **`0`**.

In [26]:
print("Cantidad de Valores Nulos de 'revenue':",df_movies['revenue'].isna().sum())
print("Cantidad de Valores Nulos de 'budget':",df_movies['budget'].isna().sum())

Cantidad de Valores Nulos de 'revenue': 6
Cantidad de Valores Nulos de 'budget': 0


In [27]:
df_movies.info()

#Efectivamente validamos que faltan 6 datos a la columna 'revenue'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45466 non-null  object 
 1   budget             45466 non-null  object 
 2   homepage           7782 non-null   object 
 3   id                 45466 non-null  object 
 4   imdb_id            45449 non-null  object 
 5   original_language  45455 non-null  object 
 6   original_title     45466 non-null  object 
 7   overview           44512 non-null  object 
 8   popularity         45461 non-null  object 
 9   poster_path        45080 non-null  object 
 10  release_date       45379 non-null  object 
 11  revenue            45460 non-null  float64
 12  runtime            45203 non-null  float64
 13  status             45379 non-null  object 
 14  tagline            20412 non-null  object 
 15  title              45460 non-null  object 
 16  video              454

In [28]:
# Se procede a sustituir los 6 valores nulos en la columna "revenue", con cero (0).
df_movies['revenue'] = df_movies['revenue'].fillna(0)

# Otra forma de hacerlo: 
# df_movies2['revenue'].fillna(0,inplace=True)

# Validamos que ya no existan valores nulos:
df_movies['revenue'].isna().sum()

0

In [29]:
### La columna 'budget' esta definida como Objeto. 
### Validamos los tipos de datos directamente en la columna:  
df_movies['budget']

0        30000000
1        65000000
2               0
3        16000000
4               0
           ...   
45461           0
45462           0
45463           0
45464           0
45465           0
Name: budget, Length: 45466, dtype: object

In [30]:
### Validando si hay NaN en la clumna 'budget:
df_movies['budget'].isna().sum()

0

In [31]:
### La columna 'budget' esta definida como Objeto, pero es Númerico.
### Convertiendo la columna de tipo objeto a tipo numerico, y los que no sean numericos los convierte en NaN:

df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')

### Validano los cambios realizados a númerico y luego si se ha creado algun NaN:
df_movies['budget'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 45466 entries, 0 to 45465
Series name: budget
Non-Null Count  Dtype  
--------------  -----  
45463 non-null  float64
dtypes: float64(1)
memory usage: 355.3 KB


In [32]:
# Validando si aparecieron NaN: (por el cambio realizado en el paso anterior)
df_movies['budget'].isna().sum()  

3

In [33]:
# Se crearon 3 valores NaN. Se procede a sustituirlos por cero (0).
df_movies['budget'] = df_movies['budget'].fillna(0)

### Validando que ya no hay NaN
df_movies['budget'].isna().sum()

0

### Requerimiento Nro. 3

+ Los valores nulos del campo **`release_date`** deben eliminarse.

In [35]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45466 non-null  object 
 1   budget             45466 non-null  float64
 2   homepage           7782 non-null   object 
 3   id                 45466 non-null  object 
 4   imdb_id            45449 non-null  object 
 5   original_language  45455 non-null  object 
 6   original_title     45466 non-null  object 
 7   overview           44512 non-null  object 
 8   popularity         45461 non-null  object 
 9   poster_path        45080 non-null  object 
 10  release_date       45379 non-null  object 
 11  revenue            45466 non-null  float64
 12  runtime            45203 non-null  float64
 13  status             45379 non-null  object 
 14  tagline            20412 non-null  object 
 15  title              45460 non-null  object 
 16  video              454

In [36]:
### La columna 'release_date' esta definida de tipo Object, segun el comando anterior.

### Validando si es o no, de tipo Objeto: (y observamos que es una fecha)
df_movies['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

In [37]:
### Validamos si tiene NaN:
df_movies['release_date'].isna().sum()

87

In [38]:
### Primero convertimos a datetime la columna 'release_date'
### Y forzamos a convertir en NaN a todos los valores que den error (errors='coerce).
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')

### Validamos nuevamente la cantidad de valores NaN
df_movies['release_date'].isna().sum()

90

In [39]:
### Ahora si procedemos a eliminar los valores NaN = 87 +3 = 90, de la columna 'release_date'.
df_movies.dropna(subset=['release_date'], inplace=True)

#Verificamos que no existan NaT
df_movies['release_date'].isnull().sum()

0

In [40]:
### Validamos nuevamente:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   adult              45376 non-null  object        
 1   budget             45376 non-null  float64       
 2   homepage           7766 non-null   object        
 3   id                 45376 non-null  object        
 4   imdb_id            45362 non-null  object        
 5   original_language  45365 non-null  object        
 6   original_title     45376 non-null  object        
 7   overview           44435 non-null  object        
 8   popularity         45376 non-null  object        
 9   poster_path        45037 non-null  object        
 10  release_date       45376 non-null  datetime64[ns]
 11  revenue            45376 non-null  float64       
 12  runtime            45130 non-null  float64       
 13  status             45296 non-null  object        
 14  tagline    

### Requerimiento Nro. 4

+ De haber fechas, deberán tener el formato **`AAAA-mm-dd`**, además deberán crear la columna **`release_year`** donde extraerán el año de la fecha de estreno.

In [41]:
### Validan el tipo de columna 'release_date' y sus datos
print(df_movies['release_date'])
df_movies['release_date'].info()


0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45460   1991-05-13
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45376, dtype: datetime64[ns]
<class 'pandas.core.series.Series'>
Index: 45376 entries, 0 to 45465
Series name: release_date
Non-Null Count  Dtype         
--------------  -----         
45376 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 709.0 KB


In [42]:
#Verificamos que todos los registros tengan el mismo formato AAAA-MM-DD
df_movies['release_date'] = pd.to_datetime(df_movies['release_date']).dt.strftime('%Y-%m-%d')

In [43]:
### Como efectivamente todos los registros de la columna 'release_date' tienen el mismo formato, 
### pasamos a crear la columna de años:

#Creamos la columna 'release_year' que va a contener el año de la columna 'release_date'
df_movies['release_year'] = pd.to_datetime(df_movies['release_date']).dt.year

### Validamos la nueva columna de 'release_year'
print(df_movies[['release_date','release_year']].head(5))
df_movies[['release_date','release_year']].tail(5)

  release_date  release_year
0   1995-10-30          1995
1   1995-12-15          1995
2   1995-12-22          1995
3   1995-12-22          1995
4   1995-02-10          1995


,release_date,release_year
45460,1991-05-13,1991
45462,2011-11-17,2011
45463,2003-08-01,2003
45464,1917-10-21,1917
45465,2017-06-09,2017


In [44]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45376 non-null  object 
 1   budget             45376 non-null  float64
 2   homepage           7766 non-null   object 
 3   id                 45376 non-null  object 
 4   imdb_id            45362 non-null  object 
 5   original_language  45365 non-null  object 
 6   original_title     45376 non-null  object 
 7   overview           44435 non-null  object 
 8   popularity         45376 non-null  object 
 9   poster_path        45037 non-null  object 
 10  release_date       45376 non-null  object 
 11  revenue            45376 non-null  float64
 12  runtime            45130 non-null  float64
 13  status             45296 non-null  object 
 14  tagline            20398 non-null  object 
 15  title              45376 non-null  object 
 16  video              45376 no

### Requerimiento Nro. 5

+ Crear la columna con el retorno de inversión, llamada **`return`** con los campos **`revenue`** y **`budget`**, dividiendo estas dos últimas **`revenue / budget`**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor **`0`**.

In [45]:
### Validando que no hay NaN, en ninguna de las dos columnas:
print(df_movies['budget'].isna().sum())
df_movies['revenue'].isna().sum()

0


0

In [46]:
### Validando datos
df_movies[['budget','revenue']].describe()

,budget,revenue
count,4.537600e+04,4.537600e+04
mean,4.232604e+06,1.123010e+07
std,1.743986e+07,6.438996e+07
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00
max,3.800000e+08,2.787965e+09


Se puede observar que ambas columnas tienen valores en ceros (0)

In [47]:
### Se realiza esta conversion para detectar la existencia de algun dato, con un posible error. (Por si acaso !!!!)
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce')

In [48]:
### Creando la columna con la formula del retorno de inversión 'return'
df_movies['return'] = df_movies['revenue']/df_movies['budget']

#df_movies['return'] = df_movies.apply(lambda x_return: 0 if x_return['budget'] == 0 else (x_return['revenue'] / x_return['budget']), axis=1)

### Eliminando valores NaN
df_movies['return'].fillna(0, inplace=True)

### Visualizando las columnas
print(df_movies[['revenue', 'budget','return']].head(10))
df_movies[['revenue', 'budget','return']].tail(10)


       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0        inf
5  187436818.0  60000000.0   3.123947
6          0.0  58000000.0   0.000000
7          0.0         0.0   0.000000
8   64350171.0  35000000.0   1.838576
9  352194034.0  58000000.0   6.072311


,revenue,budget,return
45455,0.0,0.0,0.0
45456,0.0,0.0,0.0
45457,0.0,0.0,0.0
45458,0.0,0.0,0.0
45459,0.0,0.0,0.0
45460,0.0,0.0,0.0
45462,0.0,0.0,0.0
45463,0.0,0.0,0.0
45464,0.0,0.0,0.0
45465,0.0,0.0,0.0


In [49]:
### En el caso de haber divisiones por CERO, que son colocadas como: inf ( Infinito), debemos corregirlo.

### Se procede a convertir los valores inf (INFINITOS) a CERO:
df_movies['return'] = df_movies['return'].replace(np.inf, 0)

### Visualizando las columnas nuevamente:
print(df_movies[['revenue', 'budget','return']].head(10))
df_movies[['revenue', 'budget','return']].tail(10).T

       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0   0.000000
5  187436818.0  60000000.0   3.123947
6          0.0  58000000.0   0.000000
7          0.0         0.0   0.000000
8   64350171.0  35000000.0   1.838576
9  352194034.0  58000000.0   6.072311


,45455,45456,45457,45458,45459,45460,45462,45463,45464,45465
revenue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
budget,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
return,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
### Error de division por CERO corregido.

### Validamos nuevamente el Dataset:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45376 non-null  object 
 1   budget             45376 non-null  float64
 2   homepage           7766 non-null   object 
 3   id                 45376 non-null  object 
 4   imdb_id            45362 non-null  object 
 5   original_language  45365 non-null  object 
 6   original_title     45376 non-null  object 
 7   overview           44435 non-null  object 
 8   popularity         45376 non-null  object 
 9   poster_path        45037 non-null  object 
 10  release_date       45376 non-null  object 
 11  revenue            45376 non-null  float64
 12  runtime            45130 non-null  float64
 13  status             45296 non-null  object 
 14  tagline            20398 non-null  object 
 15  title              45376 non-null  object 
 16  video              45376 no

### Requerimiento Nro. 6

+ Eliminar las siguientes 6 columnas que no serán utilizadas, **`video`**,**`imdb_id`**,**`adult`**,**`original_title`**,**`poster_path`** y **`homepage`**.

In [51]:
### Total de Columnas
print("Total de Columnas=",df_movies.columns.value_counts().sum())

### Cantidad de columnas a eliminar por el Requeriiento Nro.6
print("Nro de Columnas a eliminar el el Requerimienti Nro.6 = 6 ")

### Se procede a eliminar las 6 columnas que no serán utilizadas:
df_movies.drop(columns=['video', 'imdb_id','adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

### Columnas Restantes:
print("Columnas Restantes =",df_movies.columns.value_counts().sum())

Total de Columnas= 26
Nro de Columnas a eliminar el el Requerimienti Nro.6 = 6 
Columnas Restantes = 20


In [52]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45376 non-null  float64
 1   id                 45376 non-null  object 
 2   original_language  45365 non-null  object 
 3   overview           44435 non-null  object 
 4   popularity         45376 non-null  object 
 5   release_date       45376 non-null  object 
 6   revenue            45376 non-null  float64
 7   runtime            45130 non-null  float64
 8   status             45296 non-null  object 
 9   tagline            20398 non-null  object 
 10  title              45376 non-null  object 
 11  vote_average       45376 non-null  float64
 12  vote_count         45376 non-null  float64
 13  collection         45376 non-null  object 
 14  genre              45376 non-null  object 
 15  companies          45376 non-null  object 
 16  countries          45376 no

In [53]:
### Validando Columnas:
df_movies.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'collection', 'genre', 'companies',
       'countries', 'languages', 'release_year', 'return'],
      dtype='object')

In [54]:
df_movies

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,genre,companies,countries,languages,release_year,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Animation|Comedy|Family,Pixar Animation Studios,United States of America,English,1995,12.451801
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,,Adventure|Fantasy|Family,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,English|Français,1995,4.043035
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,Romance|Comedy,Warner Bros.|Lancaster Gate,United States of America,English,1995,0.000000
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,,Comedy|Drama|Romance,Twentieth Century Fox Film Corporation,United States of America,English,1995,5.090760
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,Comedy,Sandollar Productions|Touchstone Pictures,United States of America,English,1995,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,0.0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,NaN,Robin Hood,5.7,26.0,,Drama|Action|Romance,Westdeutscher Rundfunk (WDR)|Working Title Fil...,Canada|Germany|United Kingdom|United States of...,English,1991,0.000000
45462,0.0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,NaN,Century of Birthing,9.0,3.0,,Drama,Sine Olivia,Philippines,,2011,0.000000
45463,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,6.0,,Action|Drama|Thriller,American World Pictures,United States of America,English,2003,0.000000
45464,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,NaN,Satan Triumphant,0.0,0.0,,,Yermoliev,Russia,,1917,0.000000


### 

### 1.2) ETL al DataSets: Credicts

In [142]:
### Cargango nuevamente el DataSetd: credits.csv
#df_credits = pd.read_csv('../DataSetsPI/credits.csv')

In [55]:
### Revisando cantidad de registros, tipos de datos y posibles valores nulos
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [56]:
### Visualizando los datos
df_credits.head(5)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


### En este DataFrame de Credits, tambien necesitamos desanidar por los campos significativos de acuerdo a los requerimientos del proyecto.

### **1) Visualizando la primera columna: *`cast`***

### Reparto de la Pelicula (cast)

El DataFrame que describiste parece contener información sobre el reparto (cast) de una película. Cada registro en el DataFrame representa un miembro del reparto de la película y contiene información como el ID del reparto, el personaje que interpreta, el ID del crédito, el género, el ID del actor o actriz, el nombre del actor o actriz, el orden en el que aparece en los créditos y la ruta de acceso al perfil del actor o actriz.

Aquí hay una descripción de cada una de las columnas del DataFrame:

+ "cast_id": El ID del reparto al que pertenece el actor o actriz.
+ "character": El personaje que el actor o actriz interpreta en la película.
+ "credit_id": El ID del crédito en la base de datos de la película.
+ "gender": El género del actor o actriz (1 para mujeres, 2 para hombres).
+ "id": El ID del actor o actriz en la base de datos de la película.
+ "name": El nombre del actor o actriz. 
+ "order": El orden en el que aparece el actor o actriz en los créditos de la película.
+ "profile_path": La ruta de acceso al perfil del actor o actriz en la base de datos de la película. <Br>

 Esta información puede ser útil para analizar el reparto de una película y para hacer recomendaciones de películas basadas en preferencias de actores o personajes. Por ejemplo, un usuario puede estar interesado en ver todas las películas en las que haya actuado Tom Hanks. En este caso, se puede utilizar la columna "name" para filtrar el DataFrame y obtener una lista de todas las películas en las que actuó Tom Hanks.

In [57]:
### Tomando el primer dato de la columna 'belongs_to_collection'
df_credits['cast'][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [58]:
### La misma salida pero visualizando con el comando .loc
df_credits.loc[0][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

 Se puede observar que los datos son String (Object) pero que tienen un formato de una lista con un diccionario. En este caso vamos a Desanidar inicialmente por las claves de ⏩ `'character', 'gender', y 'name' `.  En el EDA analizaremos y validaremos estos datos a profundidad. 

### **2) Visualizando la segunda columna: *`crew`***

### Equipo de Producción (crew)

La columna "crew" que describiste parece contener información sobre el equipo de producción (crew) de una película. Cada registro en la columna representa un miembro del equipo de producción de la película y contiene información como el ID del crédito, el departamento en el que trabaja, el género, el ID del miembro del equipo de producción, el trabajo que realiza, el nombre del miembro del equipo de producción y la ruta de acceso al perfil del miembro del equipo de producción.

Aquí hay una descripción de cada una de las columnas de la columna "crew":

+   "credit_id": El ID del crédito en la base de datos de la película.
+   "department": El departamento en el que trabaja el miembro del equipo de producción (por ejemplo, dirección, guion, diseño de producción, fotografía, etc.).
+    "gender": El género del miembro del equipo de producción (1 para mujeres, 2 para hombres).
+   "id": El ID del miembro del equipo de producción en la base de datos de la película.
+   "job": El trabajo que realiza el miembro del equipo de producción (por ejemplo, director, guionista, diseñador de producción, director de fotografía, etc.).
+   "name": El nombre del miembro del equipo de producción.
+   "profile_path": La ruta de acceso al perfil del miembro del equipo de producción en la base de datos de la película.

Esta información puede ser útil para analizar el equipo de producción de una película y para hacer recomendaciones de películas basadas en preferencias de trabajos específicos o en preferencias de miembros del equipo de producción específicos. Por ejemplo, un usuario puede estar interesado en ver todas las películas que hayan sido dirigidas por un director en particular. En este caso, se puede utilizar la columna "name" y la columna "job" para filtrar el DataFrame y obtener una lista de todas las películas que hayan sido dirigidas por ese director.

In [59]:
### Tomando el primer dato de la columna 'belongs_to_collection'
df_credits['crew'][0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [60]:
### La misma salida pero visualizando con el comando .loc
df_credits.loc[0][1]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A


Se puede observar que los datos son String (Object) pero que tienen un formato de una lista con varios diccionarios. En este caso vamos a Desanidar inicialmente por las claves que considero relevantes para este proyecto ⏩ `'gender', 'job' y 'name'`. En este caso necesitamos los jobs para luego filtrar por Directores de Películas y poder tener esta campo requerido, por los momentos para la funcion Nro. 6. En el EDA analizaremos y validaremos estos datos a profundidad. 

### 

### **Se procede a crear una Funcion que nos permite desanidar ambas columnas, pasando las claves por la cual queremos desanidar.** <br>
Adicionalmente, la funcion crea las nuevas columnas con los valores desanidados. <br>
Cada nombre de columna sera una concatenacion del nombre de la columna original mas el nombre de la clave a desanidar.<br>
Nombre de la función:<Br>
`Extraer_Valor_List_Credits`

In [72]:
### Haciendo una copia para trabajar con el datasets: df
### df = df_credits.copy()   #df_credits

### 1) Desanidando la primer columna: 'cast'

In [61]:
### Se procede desanidadas la columna de 'cast', por las clave valor de: 'character', 'gender' y 'name'.

claves = ['character', 'gender', 'name']

def Extraer_Valor_List_Credits(cadena, clave):
    try:
        lista_de_diccionarios = ast.literal_eval(cadena)                                               
        if isinstance(lista_de_diccionarios, list):                                      
            lista_encontrada = [str(dic.get(clave, '')) for dic in lista_de_diccionarios if isinstance(dic, dict)]
            return '|'.join(lista_encontrada)
    except (ValueError, SyntaxError):
        pass
    return ''


for clave in claves:
    df_credits[f'cast_{clave}'] = df_credits['cast'].apply(Extraer_Valor_List_Credits, clave=clave)

df_credits

,cast,crew,id,cast_character,cast_gender,cast_name
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Woody (voice)|Buzz Lightyear (voice)|Mr. Potat...,2|2|2|2|2|2|1|0|2|1|2|1|2,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Alan Parrish|Samuel Alan Parrish / Van Pelt|Ju...,2|2|1|0|1|1|2|1|0|1|2|1|2|0|0|0|2|2|0|2|0|2|0|...,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Max Goldman|John Gustafson|Ariel Gustafson|Mar...,2|2|1|1|1|2|2,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Savannah 'Vannah' Jackson|Bernadine 'Bernie' H...,1|1|1|1|2|2|2|2|2|2,Whitney Houston|Angela Bassett|Loretta Devine|...
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,George Banks|Nina Banks|Franck Eggelhoffer|Ann...,2|1|2|1|2|0|2|2|1|1|2|1,Steve Martin|Diane Keaton|Martin Short|Kimberl...
...,...,...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,||,1|0|0,Leila Hatami|Kourosh Tahami|Elham Korda
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,Sister Angela|Homer|Crazy Woman/Virgin|Amang T...,1|0|0|2|0|0|0|0|0|0|0,Angel Aquino|Perry Dizon|Hazel Orencio|Joel To...
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,Emily Shaw|Det. Mark Winston|Jayne Ferré|Alex ...,1|2|1|2|2|2|2|2|2|0|2|2|2|2|0,Erika Eleniak|Adam Baldwin|Julie du Page|James...
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,||||,2|1|2|0|1,Iwan Mosschuchin|Nathalie Lissenko|Pavel Pavlo...


In [62]:
### VAlidacion de las nuevas columnas
df_credits.columns

Index(['cast', 'crew', 'id', 'cast_character', 'cast_gender', 'cast_name'], dtype='object')

### 2) Desanidando la segunda columna: 'crew'

In [63]:
### Se procede desanidadas la columna de 'crew', por las clave valor de: 'gender', 'job' y 'name'.

claves = ['gender', 'job', 'name']

def Extraer_Valor_List_Credits(cadena, clave):
    try:
        lista_de_diccionarios = ast.literal_eval(cadena)                                               
        if isinstance(lista_de_diccionarios, list):                                      
            lista_encontrada = [str(dic.get(clave, '')) for dic in lista_de_diccionarios if isinstance(dic, dict)]
            return '|'.join(lista_encontrada)
    except (ValueError, SyntaxError):
        pass
    return ''

for clave in claves:
    df_credits[f'crew_{clave}'] = df_credits['crew'].apply(Extraer_Valor_List_Credits, clave=clave)

df_credits

,cast,crew,id,cast_character,cast_gender,cast_name,crew_gender,crew_job,crew_name
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Woody (voice)|Buzz Lightyear (voice)|Mr. Potat...,2|2|2|2|2|2|1|0|2|1|2|1|2,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,2|2|2|2|0|1|0|2|2|2|2|2|0|0|0|2|0|0|2|2|0|0|0|...,Director|Screenplay|Screenplay|Screenplay|Scre...,John Lasseter|Joss Whedon|Andrew Stanton|Joel ...
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Alan Parrish|Samuel Alan Parrish / Van Pelt|Ju...,2|2|1|0|1|1|2|1|0|1|2|1|2|0|0|0|2|2|0|2|0|2|0|...,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,2|2|2|2|2|0|0|2|2|2|2|2|2|2|2|2,Executive Producer|Screenplay|Original Music C...,Larry J. Franco|Jonathan Hensleigh|James Horne...
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Max Goldman|John Gustafson|Ariel Gustafson|Mar...,2|2|1|1|1|2|2,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,2|2|2|2,Director|Characters|Writer|Sound Recordist,Howard Deutch|Mark Steven Johnson|Mark Steven ...
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Savannah 'Vannah' Jackson|Bernadine 'Bernie' H...,1|1|1|1|2|2|2|2|2|2,Whitney Houston|Angela Bassett|Loretta Devine|...,2|0|0|2|1|0|0|0|2|0,Director|Screenplay|Producer|Producer|Producer...,Forest Whitaker|Ronald Bass|Ronald Bass|Ezra S...
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,George Banks|Nina Banks|Franck Eggelhoffer|Ann...,2|1|2|1|2|0|2|2|1|1|2|1,Steve Martin|Diane Keaton|Martin Short|Kimberl...,2|2|1|1|2|2|2,Original Music Composer|Director of Photograph...,Alan Silvestri|Elliot Davis|Nancy Meyers|Nancy...
...,...,...,...,...,...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,||,1|0|0,Leila Hatami|Kourosh Tahami|Elham Korda,0|0|0|0|0|0|0|0|0,Director|Producer|Camera Supervisor|Script|Edi...,Hamid Nematollah|Hamid Nematollah|Farshad Moha...
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,Sister Angela|Homer|Crazy Woman/Virgin|Amang T...,1|0|0|2|0|0|0|0|0|0|0,Angel Aquino|Perry Dizon|Hazel Orencio|Joel To...,0|0|0|0|0|0,Director|Writer|Production Design|Music|Editor...,Lav Diaz|Lav Diaz|Dante Perez|Lav Diaz|Lav Dia...
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,Emily Shaw|Det. Mark Winston|Jayne Ferré|Alex ...,1|2|1|2|2|2|2|2|2|0|2|2|2|2|0,Erika Eleniak|Adam Baldwin|Julie du Page|James...,2|2|2|0|2,Director|Screenplay|Screenplay|Original Music ...,Mark L. Lester|C. Courtney Joyner|Jeffrey Gold...
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,||||,2|1|2|0|1,Iwan Mosschuchin|Nathalie Lissenko|Pavel Pavlo...,0|2,Director|Producer,Yakov Protazanov|Joseph N. Ermolieff


In [64]:
### Validacion de las nuevas columnas
df_credits.columns

Index(['cast', 'crew', 'id', 'cast_character', 'cast_gender', 'cast_name',
       'crew_gender', 'crew_job', 'crew_name'],
      dtype='object')

In [65]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cast            45476 non-null  object
 1   crew            45476 non-null  object
 2   id              45476 non-null  int64 
 3   cast_character  45476 non-null  object
 4   cast_gender     45476 non-null  object
 5   cast_name       45476 non-null  object
 6   crew_gender     45476 non-null  object
 7   crew_job        45476 non-null  object
 8   crew_name       45476 non-null  object
dtypes: int64(1), object(8)
memory usage: 3.1+ MB


## Unión de los dos dataframe en uno solo llamado: df<Br>
### La unión se realizará por la columna del codigo de la pelicula: 'id'

### 1) Validando Registros Duplicados en df_credits

In [66]:
### Total de Registros duplicados en: df_credits
df_credits[df_credits['id'].duplicated()].value_counts().sum()

44

In [67]:
### Validacion de Registros Duplicado
df_credits[df_credits['id'].duplicated()].value_counts()

cast                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [68]:
### Visualizando y validando los registros duplicados de df_credits; ordenados por el codigo de la película:
df_credits[df_credits.duplicated(subset=['id'], keep=False)].sort_values(by=['id'])

,cast,crew,id,cast_character,cast_gender,cast_name,crew_gender,crew_job,crew_name
25885,"[{'cast_id': 12, 'character': 'The Creature', ...","[{'credit_id': '52fe4380c3a36847f80590dd', 'de...",3057,The Creature|Victor Frankenstein|Caroline Fran...,2|2|1|2|2|1|0|0|2,Luke Goss|Alec Newman|Julie Delpy|William Hurt...,1|2|0|2|2|1|1|2|0|2|2|0|0|0|0|0|0|0|2|0|0|0|0|...,Novel|Screenplay|Producer|Music|Director of Ph...,Mary Shelley|Mark Kruger|James Wilberger|Roger...
25950,"[{'cast_id': 12, 'character': 'The Creature', ...","[{'credit_id': '52fe4380c3a36847f80590dd', 'de...",3057,The Creature|Victor Frankenstein|Caroline Fran...,2|2|1|2|2|1|0|0|2,Luke Goss|Alec Newman|Julie Delpy|William Hurt...,1|2|0|2|2|1|1|2|0|2|2|0|0|0|0|0|0|0|2|0|0|0|0|...,Novel|Screenplay|Producer|Music|Director of Ph...,Mary Shelley|Mark Kruger|James Wilberger|Roger...
33838,"[{'cast_id': 15, 'character': 'Chuck Barris', ...","[{'credit_id': '52fe43e2c3a36847f80760a9', 'de...",4912,Chuck Barris|Penny|Patricia Watson|Keeler|Brad...,2|1|1|2|2|2|2|2|1|2|0|1|2|1|2|0|0|1|0|2|1|1|2|...,Sam Rockwell|Drew Barrymore|Julia Roberts|Rutg...,2|2|2|2|2|2|2|2|2|2|2|1|2|1|2|0,Director|Novel|Screenplay|Producer|Executive P...,George Clooney|Chuck Barris|Charlie Kaufman|An...
5865,"[{'cast_id': 15, 'character': 'Chuck Barris', ...","[{'credit_id': '52fe43e2c3a36847f80760b5', 'de...",4912,Chuck Barris|Penny|Patricia Watson|Keeler|Brad...,2|1|1|2|2|2|2|2|1|2|0|1|2|1|2|0|0|1|0|2|1|1|2|...,Sam Rockwell|Drew Barrymore|Julia Roberts|Rutg...,2|2|2|2|1|2|2|1|2|2|0|2|2|2|2|2,Screenplay|Executive Producer|Executive Produc...,Charlie Kaufman|Bob Weinstein|Harvey Weinstein...
9165,"[{'cast_id': 11, 'character': 'Jef Costello', ...","[{'credit_id': '52fe440ac3a36847f807ee01', 'de...",5511,Jef Costello|Superintendant|Jane Lagrange|Vale...,2|2|1|1|1|0|2|2|2|2|2|2|2|2|2|2|2|2|2|2|2|2|2|...,Alain Delon|François Périer|Nathalie Delon|Cat...,2|2|2|2|2|2|0|0|0|0|0|0,Director of Photography|Producer|Screenplay|Di...,Henri Decaë|Raymond Borderie|Jean-Pierre Melvi...
...,...,...,...,...,...,...,...,...,...
25887,"[{'cast_id': 7, 'character': 'Hollander', 'cre...","[{'credit_id': '52fe4da29251416c9111ce5d', 'de...",199591,Hollander|Sara|Ryan|Blake|Chad|Danny|Antonio|l...,2|1|2|1|0|2|2|0|0|0|0|0|0,Michael Keaton|Michelle Monaghan|Andrew W. Wal...,2|2|2|2|2|1|2|2|2|0|0|1|0|0|0|0|0|0|0|0|0|0|0|...,Screenplay|Producer|Editor|Original Music Comp...,David Loughery|David Loughery|Andrew Mondshein...
24163,"[{'cast_id': 2, 'character': 'Ebba', 'credit_i...","[{'credit_id': '534fd1a80e0a267eb6000e32', 'de...",265189,Ebba|Tomas|Vera|Harry|Brady|Mats|Fanni|Charlot...,1|2|0|0|2|2|0|0|0,Lisa Loven Kongsli|Johannes Bah Kuhnke|Clara W...,2|2|2|0|0|0|0|2|0|0|0|0|0|0|0,Director|Writer|Producer|Producer|Producer|Co-...,Ruben Östlund|Ruben Östlund|Philippe Bober|Eri...
45275,"[{'cast_id': 2, 'character': 'Ebba', 'credit_i...","[{'credit_id': '534fd1a80e0a267eb6000e32', 'de...",265189,Ebba|Tomas|Vera|Harry|Brady|Mats|Fanni|Charlot...,1|2|0|0|2|2|0|0|0,Lisa Loven Kongsli|Johannes Bah Kuhnke|Clara W...,2|2|2|0|0|0|0|2|0|0|0|0|0|0|0,Director|Writer|Producer|Producer|Producer|Co-...,Ruben Östlund|Ruben Östlund|Philippe Bober|Eri...
33196,"[{'cast_id': 1, 'character': 'Jenjira', 'credi...","[{'credit_id': '5448c8efc3a3680fb4001582', 'de...",298721,Jenjira|Itt|Keng|Nurse Tet|The Mediator|Librar...,0|0|0|0|0|0|0,Jenjira Pongpas|Banlop Lomnoi|Jarinpattra Ruea...,2|2|2|0|2|2|0|2|0|0|0|0|0|0|2|0,Director|Screenplay|Producer|Producer|Director...,Apichatpong Weerasethakul|Apichatpong Weeraset...


In [70]:
### Duplicados de df_credits
duplicados = df_credits[df_credits.duplicated(subset=['id'], keep=False)].sort_values(by=['id'])
#df_movies.loc[df_movies.duplicated(subset=['id'], keep=False), :]
print(duplicados)

                                                    cast  \
25885  [{'cast_id': 12, 'character': 'The Creature', ...   
25950  [{'cast_id': 12, 'character': 'The Creature', ...   
33838  [{'cast_id': 15, 'character': 'Chuck Barris', ...   
5865   [{'cast_id': 15, 'character': 'Chuck Barris', ...   
9165   [{'cast_id': 11, 'character': 'Jef Costello', ...   
...                                                  ...   
25887  [{'cast_id': 7, 'character': 'Hollander', 'cre...   
24163  [{'cast_id': 2, 'character': 'Ebba', 'credit_i...   
45275  [{'cast_id': 2, 'character': 'Ebba', 'credit_i...   
33196  [{'cast_id': 1, 'character': 'Jenjira', 'credi...   
40051  [{'cast_id': 1, 'character': 'Jenjira', 'credi...   

                                                    crew      id  \
25885  [{'credit_id': '52fe4380c3a36847f80590dd', 'de...    3057   
25950  [{'credit_id': '52fe4380c3a36847f80590dd', 'de...    3057   
33838  [{'credit_id': '52fe43e2c3a36847f80760a9', 'de...    4912   
5865   

### 2) Validando Registros Duplicados en df_movies

In [71]:
### Total de Registros duplicados en: df_movies
df_movies[df_movies['id'].duplicated()].value_counts().sum()

11

In [72]:
### Visualizando los registros duplicados de df_movies
df_movies[df_movies['id'].duplicated()].value_counts()

budget      id      original_language  overview                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             popularity  release_date  revenue     runtime  status    tagline                                                                                                            

In [73]:
### Visualizando y validando los registros duplicados de df_movies; ordenados por el codigo de la película:
df_movies[df_movies.duplicated(subset=['id'], keep=False)].sort_values(by=['id'])

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,genre,companies,countries,languages,release_year,return
676,0.0,105045,de,"East-Berlin, 1961, shortly after the erection ...",0.122178,1995-02-16,0.0,115.0,Released,"A love, a hope, a wall.",The Promise,5.0,1.0,,Drama|Romance,Studio Babelsberg|Centre National de la Cinéma...,Germany,Deutsch,1995,0.000000
1465,0.0,105045,de,"East-Berlin, 1961, shortly after the erection ...",0.122178,1995-02-16,0.0,115.0,Released,"A love, a hope, a wall.",The Promise,5.0,1.0,,Drama|Romance,Studio Babelsberg|Centre National de la Cinéma...,Germany,Deutsch,1995,0.000000
44821,16000000.0,10991,ja,When Molly Hale's sadness of her father's disa...,6.480376,2000-07-08,68411275.0,93.0,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,144.0,Pokémon Collection,Adventure|Fantasy|Animation|Action|Family,TV Tokyo|4 Kids Entertainment|Nintendo|Pikachu...,Japan,English,2000,4.275705
4114,16000000.0,10991,ja,When Molly Hale's sadness of her father's disa...,10.264597,2000-07-08,68411275.0,93.0,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,143.0,Pokémon Collection,Adventure|Fantasy|Animation|Action|Family,TV Tokyo|4 Kids Entertainment|Nintendo|Pikachu...,Japan,English,2000,4.275705
5710,0.0,109962,en,Two literary women compete for 20 years: one w...,12.180836,1981-09-23,0.0,115.0,Released,"From the very beginning, they knew they'd be f...",Rich and Famous,4.9,7.0,,Drama,Metro-Goldwyn-Mayer (MGM)|Jaquet,United States of America,English,1981,0.000000
20899,0.0,109962,en,Two literary women compete for 20 years: one w...,10.396878,1981-09-23,0.0,115.0,Released,"From the very beginning, they knew they'd be f...",Rich and Famous,4.9,7.0,,Drama,Metro-Goldwyn-Mayer (MGM)|Jaquet,United States of America,English,1981,0.000000
23534,3512454.0,110428,fr,"Winter, 1915. Confined by her family to an asy...",0.110065,2013-03-13,115860.0,95.0,Released,NaN,Camille Claudel 1915,7.0,20.0,,Drama,Canal+|Arte France Cinéma|3B Productions|C.R.R...,France,Français,2013,0.032985
4356,3512454.0,110428,fr,"Winter, 1915. Confined by her family to an asy...",0.134014,2013-03-13,115860.0,95.0,Released,NaN,Camille Claudel 1915,7.0,20.0,,Drama,Canal+|Arte France Cinéma|3B Productions|C.R.R...,France,Français,2013,0.032985
24844,0.0,11115,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,2008-01-29,0.0,85.0,Released,NaN,Deal,5.2,22.0,,Comedy|Drama,Andertainment Group|Crescent City Pictures|Tag...,United States of America,English,2008,0.000000
14012,0.0,11115,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,2008-01-29,0.0,85.0,Released,NaN,Deal,5.2,22.0,,Comedy|Drama,Andertainment Group|Crescent City Pictures|Tag...,United States of America,English,2008,0.000000


### 

In [74]:
### Visualizando los registros duplicados de df_movies
duplicados = df_movies[df_movies.duplicated(subset=['id'], keep=False)].sort_values(by=['id'])
#df_movies.loc[df_movies.duplicated(subset=['id'], keep=False), :]
print(duplicados)

           budget      id original_language  \
676           0.0  105045                de   
1465          0.0  105045                de   
44821  16000000.0   10991                ja   
4114   16000000.0   10991                ja   
5710          0.0  109962                en   
20899         0.0  109962                en   
23534   3512454.0  110428                fr   
4356    3512454.0  110428                fr   
24844         0.0   11115                en   
14012         0.0   11115                en   
21165         0.0  119916                en   
19890         0.0  119916                en   
5535          0.0   12600                ja   
44826         0.0   12600                ja   
11342      2500.0   13209                fa   
15765      2500.0   13209                fa   
838           0.0  132641                ja   
30001         0.0  132641                ja   
16764         0.0  141971                fi   
13261         0.0  141971                fi   
13375        

### Dado que son tan pequeños el número de registros duplicados en cada datasets, se procede a eliminarlos:

In [75]:
df_movies.shape

(45376, 20)

In [76]:
df_credits.shape

(45476, 9)

In [77]:
### Total Registros Duplicados en df_credits:
print("Total Duplicados en df_credits", df_credits[df_credits['id'].duplicated()].value_counts().sum())

### Eliminando los valores duplicados en df_credits
df_credits.drop_duplicates(subset=['id'], keep='first', inplace=True)
df_credits[df_credits['id'].duplicated()].value_counts().sum()

Total Duplicados en df_credits 44


0

In [78]:
### Total Registros Duplicados en df_movies:
print("Total Duplicados en df_movies", df_movies[df_movies['id'].duplicated()].value_counts().sum())

### Eliminando los valores duplicados en df_movies
df_movies.drop_duplicates(subset=['id'], keep='first', inplace=True)
df_movies[df_movies['id'].duplicated()].value_counts().sum()

Total Duplicados en df_movies 11


0

In [79]:
### Validando los registros eliminados de df_movies
df_movies.shape

(45346, 20)

In [80]:
### Validando los registros eliminados de df_credits
df_credits.shape

(45432, 9)

In [81]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 45475
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cast            45432 non-null  object
 1   crew            45432 non-null  object
 2   id              45432 non-null  int64 
 3   cast_character  45432 non-null  object
 4   cast_gender     45432 non-null  object
 5   cast_name       45432 non-null  object
 6   crew_gender     45432 non-null  object
 7   crew_job        45432 non-null  object
 8   crew_name       45432 non-null  object
dtypes: int64(1), object(8)
memory usage: 3.5+ MB


In [82]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45346 non-null  float64
 1   id                 45346 non-null  object 
 2   original_language  45335 non-null  object 
 3   overview           44405 non-null  object 
 4   popularity         45346 non-null  object 
 5   release_date       45346 non-null  object 
 6   revenue            45346 non-null  float64
 7   runtime            45100 non-null  float64
 8   status             45266 non-null  object 
 9   tagline            20387 non-null  object 
 10  title              45346 non-null  object 
 11  vote_average       45346 non-null  float64
 12  vote_count         45346 non-null  float64
 13  collection         45346 non-null  object 
 14  genre              45346 non-null  object 
 15  companies          45346 non-null  object 
 16  countries          45346 no

### Se procede a la union de los dos dataframes:

In [83]:
### Se lleva en ambas columnas el tipo de codigo de la película 'id' a ent64 para facilitar el merge:
df_credits['id'] = df_credits['id'].astype('int64')
df_movies['id'] = df_movies['id'].astype('int64')

In [84]:
### Se procede a realizar el merge entre los dos dataframes:
df = pd.merge(df_movies, df_credits, on='id')
df

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,release_year,return,cast,crew,cast_character,cast_gender,cast_name,crew_gender,crew_job,crew_name
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,1995,12.451801,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",Woody (voice)|Buzz Lightyear (voice)|Mr. Potat...,2|2|2|2|2|2|1|0|2|1|2|1|2,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,2|2|2|2|0|1|0|2|2|2|2|2|0|0|0|2|0|0|2|2|0|0|0|...,Director|Screenplay|Screenplay|Screenplay|Scre...,John Lasseter|Joss Whedon|Andrew Stanton|Joel ...
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,1995,4.043035,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Alan Parrish|Samuel Alan Parrish / Van Pelt|Ju...,2|2|1|0|1|1|2|1|0|1|2|1|2|0|0|0|2|2|0|2|0|2|0|...,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,2|2|2|2|2|0|0|2|2|2|2|2|2|2|2|2,Executive Producer|Screenplay|Original Music C...,Larry J. Franco|Jonathan Hensleigh|James Horne...
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,1995,0.000000,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",Max Goldman|John Gustafson|Ariel Gustafson|Mar...,2|2|1|1|1|2|2,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,2|2|2|2,Director|Characters|Writer|Sound Recordist,Howard Deutch|Mark Steven Johnson|Mark Steven ...
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,1995,5.090760,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",Savannah 'Vannah' Jackson|Bernadine 'Bernie' H...,1|1|1|1|2|2|2|2|2|2,Whitney Houston|Angela Bassett|Loretta Devine|...,2|0|0|2|1|0|0|0|2|0,Director|Screenplay|Producer|Producer|Producer...,Forest Whitaker|Ronald Bass|Ronald Bass|Ezra S...
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,1995,0.000000,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",George Banks|Nina Banks|Franck Eggelhoffer|Ann...,2|1|2|1|2|0|2|2|1|1|2|1,Steve Martin|Diane Keaton|Martin Short|Kimberl...,2|2|1|1|2|2|2,Original Music Composer|Director of Photograph...,Alan Silvestri|Elliot Davis|Nancy Meyers|Nancy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45340,0.0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,NaN,...,1991,0.000000,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de...",Sir Robert Hode|Maid Marian|Little John|Sir Mi...,2|1|2|2|0,Patrick Bergin|Uma Thurman|David Morrissey|Jür...,2|0|2|0|1|0|0|2|1,Director|Writer|Writer|Story|Producer|Music|Di...,John Irvin|Sam Resnick|John McGrath|Sam Resnic...
45341,0.0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,NaN,...,2011,0.000000,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",Sister Angela|Homer|Crazy Woman/Virgin|Amang T...,1|0|0|2|0|0|0|0|0|0|0,Angel Aquino|Perry Dizon|Hazel Orencio|Joel To...,0|0|0|0|0|0,Director|Writer|Production Design|Music|Editor...,Lav Diaz|Lav Diaz|Dante Perez|Lav Diaz|Lav Dia...
45342,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,2003,0.000000,"[{

In [85]:
df.shape

(45345, 28)

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45345 entries, 0 to 45344
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45345 non-null  float64
 1   id                 45345 non-null  int64  
 2   original_language  45334 non-null  object 
 3   overview           44404 non-null  object 
 4   popularity         45345 non-null  object 
 5   release_date       45345 non-null  object 
 6   revenue            45345 non-null  float64
 7   runtime            45099 non-null  float64
 8   status             45265 non-null  object 
 9   tagline            20387 non-null  object 
 10  title              45345 non-null  object 
 11  vote_average       45345 non-null  float64
 12  vote_count         45345 non-null  float64
 13  collection         45345 non-null  object 
 14  genre              45345 non-null  object 
 15  companies          45345 non-null  object 
 16  countries          453

In [87]:
df.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'collection', 'genre', 'companies',
       'countries', 'languages', 'release_year', 'return', 'cast', 'crew',
       'cast_character', 'cast_gender', 'cast_name', 'crew_gender', 'crew_job',
       'crew_name'],
      dtype='object')

### Se procede a reorganizar las 28 columnas del nuevo dataframe

In [88]:
### Se procede a reorganizar las 28 columnas
df = df.reindex(columns=['id', 'title', 'crew_job','crew_name','collection','genre','release_date','release_year','budget','revenue','original_language','languages','overview','popularity','runtime','status','tagline','vote_average','vote_count','companies','countries','return','cast_character','cast_gender','cast_name','crew_gender','crew','cast'])

In [89]:
df.columns

Index(['id', 'title', 'crew_job', 'crew_name', 'collection', 'genre',
       'release_date', 'release_year', 'budget', 'revenue',
       'original_language', 'languages', 'overview', 'popularity', 'runtime',
       'status', 'tagline', 'vote_average', 'vote_count', 'companies',
       'countries', 'return', 'cast_character', 'cast_gender', 'cast_name',
       'crew_gender', 'crew', 'cast'],
      dtype='object')

### Se realiza Backup de los datasets modificados

In [90]:
### Se realiza un Backup en CSV, del DataSet df_movies con los cambios relizados por requerimientos iniciales (df_movies-OK.csv)
df_movies.to_csv('../DataSetPI-Cleaning/df_movies-OK.csv', index=False, header=True, sep=';')

### Se realiza un Backup en CSV, del DataSet df_credits con los cambios relizados por requerimientos iniciales (df_movies-OK.csv)
df_credits.to_csv('../DataSetPI-Cleaning/df_movies-OK.csv', index=False, header=True, sep=';')

### Se realiza un Backup en CSV, del nuevo DataSet "df" con los cambios realizados (df_merged-OK.csv)
df.to_csv('../DataSetPI-Cleaning/df_merged-OK.csv', index=False, header=True, sep=';')

### Se realiza un Backup en Excel, del DataSet con los cambios relizados por requerimientos iniciales (df_merged-OK.xlsx)
df.to_excel('../DataSetPI-Cleaning/df_merged-OK.xlsx', index=False, header=True)

In [192]:
#df_credits = pd.read_csv('../DataSetsPI/credits.csv')
df = pd.read_csv('../DataSetPI-Cleaning/df_merged-OK.csv', sep=';')

In [194]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45345 entries, 0 to 45344
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 45345 non-null  int64  
 1   title              45345 non-null  object 
 2   crew_job           44622 non-null  object 
 3   crew_name          44622 non-null  object 
 4   collection         4485 non-null   object 
 5   genre              42962 non-null  object 
 6   release_date       45345 non-null  object 
 7   release_year       45345 non-null  int64  
 8   budget             45345 non-null  float64
 9   revenue            45345 non-null  float64
 10  original_language  45334 non-null  object 
 11  languages          41457 non-null  object 
 12  overview           44404 non-null  object 
 13  popularity         45345 non-null  float64
 14  runtime            45099 non-null  float64
 15  status             45265 non-null  object 
 16  tagline            203

# Finalización ETL 

### 

### 

# Inicio de las Funciones APis en FastAPI

### 

In [91]:
df.columns

Index(['id', 'title', 'crew_job', 'crew_name', 'collection', 'genre',
       'release_date', 'release_year', 'budget', 'revenue',
       'original_language', 'languages', 'overview', 'popularity', 'runtime',
       'status', 'tagline', 'vote_average', 'vote_count', 'companies',
       'countries', 'return', 'cast_character', 'cast_gender', 'cast_name',
       'crew_gender', 'crew', 'cast'],
      dtype='object')

In [114]:
df.shape

(45345, 28)

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45345 entries, 0 to 45344
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 45345 non-null  int64  
 1   title              45345 non-null  object 
 2   crew_job           45345 non-null  object 
 3   crew_name          45345 non-null  object 
 4   collection         45345 non-null  object 
 5   genre              45345 non-null  object 
 6   release_date       45345 non-null  object 
 7   release_year       45345 non-null  int32  
 8   budget             45345 non-null  float64
 9   revenue            45345 non-null  float64
 10  original_language  45334 non-null  object 
 11  languages          45345 non-null  object 
 12  overview           44404 non-null  object 
 13  popularity         45345 non-null  object 
 14  runtime            45099 non-null  float64
 15  status             45265 non-null  object 
 16  tagline            203

## Creacion de las 6 Funciones Requeridas

### Función Nro. 1

'''Ingresas el idioma, retornando la cantidad de peliculas producidas en el mismo'''

In [109]:
### Codigos de Lenguajes para probar la función:
df['original_language'].unique()

array(['en', 'fr', 'zh', 'it', 'fa', 'nl', 'de', 'cn', 'ar', 'es', 'ru',
       'sv', 'ja', 'ko', 'sr', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy',
       'vi', 'cs', 'da', 'no', 'nb', 'pl', 'el', 'sh', 'xx', 'mk', 'bo',
       'ca', 'fi', 'th', 'sk', 'bs', 'hi', 'tr', 'is', 'ps', 'ab', 'eo',
       'ka', 'mn', 'bm', 'zu', 'uk', 'af', 'la', 'et', 'ku', 'fy', 'lv',
       'ta', 'sl', 'tl', 'ur', 'rw', 'id', 'bg', 'mr', 'lt', 'kk', 'ms',
       'sq', nan, 'qu', 'te', 'am', 'jv', 'tg', 'ml', 'hr', 'lo', 'ay',
       'kn', 'eu', 'ne', 'pa', 'ky', 'gl', 'uz', 'sm', 'mt', 'hy', 'iu',
       'lb', 'si'], dtype=object)

In [111]:
### Función Nro. 1
#@app.get('/peliculas_idioma/{idioma}')
def peliculas_idioma(idioma:str):
    '''Ingresas el idioma, retornando la cantidad de peliculas producidas en el mismo'''
    try:
        mask = df[df['original_language'] == idioma]
        respuesta = int(mask['original_language'].count())
    except (ValueError, SyntaxError):
        pass 
    return {'idioma':idioma, 'cantidad':respuesta}


### Probando la funcón::
Idioma = 'es'
Respuesta = peliculas_idioma(Idioma)
print(Respuesta)  

{'idioma': 'es', 'cantidad': 991}


### Función Nro. 2

'''Ingresas la pelicula, retornando la duracion y el año'''

In [108]:
### Titulos de Películas para probar la función:
df['title'][0:5]

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

In [113]:
### PROGRAMA Nro. 2
#@app.get('/peliculas_duracion/{pelicula}')   
def peliculas_duracion(pelicula:str):
    '''Ingresas la pelicula, retornando la duracion y el año'''
    try:
        duracion = df[df['title'] == pelicula]['runtime']
        anio = int(df[df['title'] == pelicula]['release_year'])
    except (ValueError, SyntaxError):
        pass 
    return {'pelicula':pelicula, 'duracion':duracion, 'anio':anio}

### Llamando a la función:
Pelicula = 'Jumanji'
Respuesta = peliculas_duracion(Pelicula)
print(Respuesta)  

{'pelicula': 'Jumanji', 'duracion': 1    104.0
Name: runtime, dtype: float64, 'anio': 1995}


C:\Users\lesme\AppData\Local\Temp\ipykernel_1820\4091031253.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  anio = int(df[df['title'] == pelicula]['release_year'])


### Función Nro. 3

'''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio'''

In [114]:
### Visualizando Franquicias para probar la función:
df['collection'].unique()

array(['Toy Story Collection', '', 'Grumpy Old Men Collection', ...,
       'Ducobu Collection', 'Mister Blot Collection',
       'Red Lotus Collection'], dtype=object)

In [206]:
#Franquicia = df_movies[df_movies['collection'] == 'Toy Story Collection']['title'].count()
# 'Toy Story Collection'

### PROGRAMA Nro. 3
#@app.get('/franquicia/{franquicia}')
def franquicia(franquicia:str):
    '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio'''
    try:
        cantidad = df[df['collection'] == franquicia]['title'].count()
        ganancia_total = df[df['collection'] == franquicia]['revenue'].sum()
        if cantidad == 0:
            ganancia_promedio = 0
        else:
            ganancia_promedio = ganancia_total/cantidad
    except (ValueError, SyntaxError):
        pass 
    return {'franquicia':franquicia, 'cantidad':cantidad, 'ganancia_total':ganancia_total, 'ganancia_promedio':ganancia_promedio}

### Llamando a la función:
franquicia1 = 'Ducobu Collection'
Respuesta = franquicia(franquicia1)
print(Respuesta)  

{'franquicia': 'Ducobu Collection', 'cantidad': 1, 'ganancia_total': 0.0, 'ganancia_promedio': 0.0}


### Función Nro. 4

'''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo'''

In [122]:
### Visualizando Paises para probar la función:
df['countries'].unique()

array(['United States of America', 'Germany|United States of America',
       'United Kingdom|United States of America', ...,
       'Poland|Czech Republic|Slovakia', 'Cuba|Germany|Spain',
       'Egypt|Italy|United States of America'], dtype=object)

In [205]:
### PROGRAMA Nro. 4    
#@app.get('/peliculas_pais/{pais}')
def peliculas_pais(pais:str):
    '''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo'''
    try:
        respuesta = int(df[df['countries'] == pais]['title'].count())
    except (ValueError, SyntaxError):
        pass 
    return {'pais':pais, 'cantidad':respuesta}


### Llamando a la función:
Pais = 'Spain'
Respuesta = peliculas_pais(Pais)
print(Respuesta)  

{'pais': 'Spain', 'cantidad': 396}


### Función Nro. 5

'''Ingresas la productora, entregandote el revenue total y la cantidad de peliculas que realizo '''

In [138]:
### Visualizando Compañias Productoras para visualizar la función:
df['companies'].unique()

array(['Pixar Animation Studios',
       'TriStar Pictures|Teitler Film|Interscope Communications',
       'Warner Bros.|Lancaster Gate', ...,
       'Westdeutscher Rundfunk (WDR)|Working Title Films|20th Century Fox Television|CanWest Global Communications',
       'Sine Olivia', 'Yermoliev'], dtype=object)

In [204]:
### PROGRAMA Nro. 5
def productoras_exitosas(productora:str):
    '''Ingresas la productora, entregandote el revenue total y la cantidad de peliculas que realizo '''
    try:
        revenue_total = float(df[df['companies'] == productora]['revenue'].sum())
        cantidad = int(df[df['companies'] == productora]['revenue'].count())
    except (ValueError, SyntaxError):
        pass 
    return {'productora':productora, 'revenue_total': revenue_total,'cantidad':cantidad}

Productora = 'Pixar Animation Studios'
Respuesta = productoras_exitosas(Productora)
print(Respuesta)

{'productora': 'Pixar Animation Studios', 'revenue_total': 2546355520.0, 'cantidad': 28}


### Función Nro. 6

''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. <Br>
Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma. En formato Lista '''

In [142]:
### Visualizando nombres de directores para probar la función:
df['crew_name']

0        John Lasseter|Joss Whedon|Andrew Stanton|Joel ...
1        Larry J. Franco|Jonathan Hensleigh|James Horne...
2        Howard Deutch|Mark Steven Johnson|Mark Steven ...
3        Forest Whitaker|Ronald Bass|Ronald Bass|Ezra S...
4        Alan Silvestri|Elliot Davis|Nancy Meyers|Nancy...
                               ...                        
45340    John Irvin|Sam Resnick|John McGrath|Sam Resnic...
45341    Lav Diaz|Lav Diaz|Dante Perez|Lav Diaz|Lav Dia...
45342    Mark L. Lester|C. Courtney Joyner|Jeffrey Gold...
45343                 Yakov Protazanov|Joseph N. Ermolieff
45344                                        Daisy Asquith
Name: crew_name, Length: 45345, dtype: object

In [195]:
### Cantidad de nulos en la columna "crew_name":
df['crew_name'].isna().sum()

723

In [196]:
df[df['crew_name'].isna()]

,id,title,crew_job,crew_name,collection,genre,release_date,release_year,budget,revenue,...,vote_count,companies,countries,return,cast_character,cast_gender,cast_name,crew_gender,crew,cast
189,56088,The Show,NaN,NaN,NaN,Documentary,1995-08-25,1995,0.0,0.0,...,2.0,NaN,NaN,0.0,Himself|Himself|Himself|Himself|Himself|Himsel...,0|2|2|2|2|2|0|0,Kurtis Blow|Sean Combs|Snoop Dogg|Dr. Dre|Warr...,NaN,[],"[{'cast_id': 4, 'character': 'Himself', 'credi..."
614,123505,A Modern Affair,NaN,NaN,NaN,NaN,1996-09-06,1996,0.0,0.0,...,1.0,NaN,NaN,0.0,Grace Rhodes|Peter Kessler|Elaine|Lindsey|Erne...,1|2|1|0|2|2|0|1|1|2|0,Lisa Eichhorn|Stanley Tucci|Caroline Aaron|Mar...,NaN,[],"[{'cast_id': 1, 'character': 'Grace Rhodes', '..."
635,339428,Happy Weekend,NaN,NaN,NaN,Comedy,1996-03-14,1996,0.0,65335.0,...,0.0,Senator Film Produktion,Germany,0.0,Joachim Krippo|Horst|Opa Krippo|Uschi|Engel|Su...,0|2|0|0|2|1|0,Erik Goertz|Anton Rattinger|Dieter Wardetzky|I...,NaN,[],"[{'cast_id': 0, 'character': 'Joachim Krippo',..."
661,318177,Tarantella,NaN,NaN,NaN,NaN,1996-03-15,1996,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[],[]
861,253632,Baton Rouge,NaN,NaN,NaN,Thriller,1988-10-08,1988,0.0,0.0,...,0.0,NaN,NaN,0.0,Ana Alonzo|Antonio|Isabel,1|2|1,Victoria Abril|Antonio Banderas|Carmen Maura,NaN,[],"[{'cast_id': 1, 'character': 'Ana Alonzo', 'cr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45044,51078,Getting Out,NaN,NaN,NaN,Drama|Thriller,1994-04-25,1994,0.0,0.0,...,2.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[],[]
45109,332543,A Genius Leaves the Hood - The Unauthorized St...,NaN,NaN,NaN,Documentary,2014-03-21,2014,0.0,0.0,...,2.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,[],[]
45134,28469,Der Herr Karl,NaN,NaN,NaN,Comedy,1961-11-15,1961,0.0,0.0,...,1.0,Österreichischer Rundfunk (ORF),Austria,0.0,Herr Karl,2,Helmut Qualtinger,NaN,[],"[{'cast_id': 1, 'character': 'Herr Karl', 'cre..."
45149,458618,Ivanka Trump- America's Real First Lady?,NaN,NaN,NaN,Documentary|TV Movie,2017-05-15,2017,0.0,0.0,...,1.0,NaN,NaN,0.0,Herself|Himself - Host,0|0,Ivanka Trump|Matt Frei,NaN,[],"[{'cast_id': 0, 'character': 'Herself', 'credi..."


In [197]:
### Procedemos a eliminar esta filas de valores NaN  de la columna "crew_name" ya que vamos a estar haciendo busqueda y math
### por esta columna en la función numero 6.
df = df.dropna(subset=['crew_name'])

### Validando que se eliminaron los valores nulos de la clolumna "crew_name"
### Cantidad de nulos en la columna "crew_name":
df['crew_name'].isna().sum()

0

In [198]:
### Se realiza un Backup en CSV, del nuevo DataSet "df" con los cambios realizados (df_merged-OK.csv)
df.to_csv('../DataSetPI-Cleaning/df_merged-OK.csv', index=False, header=True, sep=';')

In [199]:
df = pd.read_csv('../DataSetPI-Cleaning/df_merged-OK.csv', sep=';')

In [200]:
df['crew_name'].isna().sum()

0

In [201]:
### FUNCION Nro. 6
#@app.get('/get_director/{nombre_director}')
def get_director(nombre_director:str):
    try:
        # Buscar todas las filas que contienen el nombre del director en la columna "crew_name"
        PeliculasPorDirector = df[df['crew_name'].str.contains(nombre_director)]

        # Buscar todas las filas que contienen el trabajo de director en la columna "crew_job"
        PeliculasPorCargo = PeliculasPorDirector[PeliculasPorDirector['crew_job'].str.contains('Director')]

        # Seleccionar solo las columnas "title" y "release_year"
        PeliculasPor_TituloyAnio = PeliculasPorCargo.loc[:, ['title', 'release_year', 'return','budget', 'revenue']]

        # Convertir el DataFrame en una lista de diccionarios
        Lista_De_Dicc = PeliculasPor_TituloyAnio.to_dict('records')

        # Convertir la lista de diccionarios a formato JSON
        Lista_De_Dicc_a_json = json.dumps(Lista_De_Dicc)
        
    except (ValueError, SyntaxError):
        pass 
    return  Lista_De_Dicc

### Llamando a la función:
NombreDelDirector = 'Forest Whitaker'
Respuesta = get_director(NombreDelDirector)
print(Respuesta)  




[{'title': 'Waiting to Exhale', 'release_year': 1995, 'return': 5.09075975, 'budget': 16000000.0, 'revenue': 81452156.0}, {'title': 'Hope Floats', 'release_year': 1998, 'return': 0.0, 'budget': 0.0, 'revenue': 0.0}, {'title': 'First Daughter', 'release_year': 2004, 'return': 0.0, 'budget': 30000000.0, 'revenue': 0.0}, {'title': 'Powder Blue', 'release_year': 2009, 'return': 0.0, 'budget': 0.0, 'revenue': 0.0}, {'title': 'Monte Carlo', 'release_year': 2011, 'return': 0.87125, 'budget': 20000000.0, 'revenue': 17425000.0}, {'title': 'Strapped', 'release_year': 1993, 'return': 0.0, 'budget': 0.0, 'revenue': 0.0}, {'title': 'Fruitvale Station', 'release_year': 2013, 'return': 0.0, 'budget': 0.0, 'revenue': 17385830.0}, {'title': 'Serving Life', 'release_year': 2011, 'return': 0.0, 'budget': 0.0, 'revenue': 0.0}]


# FIN DEL ETL

### 

# INICIO DEL EDA